In [3]:
import os 
path = "/home/mr/Grive/01.ALDA/Project-Alda/data"
import math
import numpy as np

In [4]:
ratings = sc.textFile(os.path.join(path, 'ratings.csv')) \
    .map(lambda line: line.split(",")) \
    .filter(lambda line: len(line)>1) \
    .map(lambda a: (a[0], a[1], a[2]))

header = ratings.first()
ratings = ratings.filter(lambda line: line != header)
ratings = ratings.map(lambda x:(int(x[0]), int(x[1]), float(x[2])))

ratings2 = ratings.map(lambda x: ((x[0], x[1]), x[2]))
user_ratings = ratings.map(lambda x:(x[0], x[2])).groupByKey()
user_ratings_mean_std = user_ratings.map(lambda x:(x[0], (np.mean(list(x[1])), np.std(list(x[1])))))
normalized_ratings = ratings.map(lambda x:(x[0], (x[1], x[2])))\
                                .join(user_ratings_mean_std)\
                                .map(lambda x:(x[0], x[1][0][0], (x[1][0][1]-x[1][1][0])/x[1][1][1]))
print normalized_ratings.take(1)

[(512, 318, 1.0864978270065191)]


In [5]:
movies = sc.textFile(os.path.join(path, 'movies.csv'))\
        .map(lambda line:line.split(","))\
        .filter(lambda line:len(line)>1)\
        .map(lambda a:(a[0],a[1]))
header2 = movies.first()
movies = movies.filter(lambda line: line != header2)
movie_indices = movies.map(lambda x:int(x[0])).distinct()

In [6]:
movie_user = normalized_ratings.map(lambda x:(x[1], x[0])).groupByKey()
user_movie = normalized_ratings.map(lambda x:(x[0], x[1])).groupByKey()
user_movie_rating = normalized_ratings.map(lambda x:((x[0], x[1]), x[2]))

In [7]:
def get_rating(user_index, movie_index, k):
    
    watched_users = movie_user.lookup(movie_index)
    watched_users = list(watched_users[0])
    if len(watched_users) == 0:
        print ("No one watched this movie yet\n")
    #print len(watched_users)
    user_watched = list(user_movie.lookup(user_index)[0])
    if movie_index in user_watched:
        return ratings2.lookup((user_index, movie_index))[0]
    sim_measures = []
    for cur_user in watched_users:
        dist = 0.0
        cur_user_watched = list(user_movie.lookup(cur_user)[0])
        common_watched = [val for val in user_watched if val in cur_user_watched]
        if len(common_watched) == 0:
            continue
        n = len(common_watched)
        for movie in common_watched:
            u_rate = user_movie_rating.lookup((user_index, movie))[0]
            cur_u_rate = user_movie_rating.lookup((cur_user, movie))[0]
            dist+=((u_rate-cur_u_rate)**2)
        temp = (dist/n, (cur_user, user_movie_rating.lookup((cur_user, movie_index))[0]))
        sim_measures.append(temp)
    sim_measures_rdd = sc.parallelize(sim_measures)
    nearestk = sim_measures_rdd.takeOrdered(k)
    avg_rating = 0
    for i in nearestk:
        avg_rating += i[0]
    avg_rating /= k
    user_mean_std = user_ratings_mean_std.lookup(user_index)[0]
    #print user_mean_std
    return (avg_rating*user_mean_std[1])+user_mean_std[0]
            
        
        
    

In [8]:
print get_rating(1, 10, 3)

2.55186514396


In [9]:
print get_rating(1, 23, 3)

2.60923175019
